In [1]:
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from sklearn import svm
from sklearn import linear_model

In [2]:
def parseDataFromFile(fname):
    for l in open(fname):
        yield eval(l)
data = list(parseDataFromFile('beer_50000.json'))
data[46]

{'review/appearance': 4.0,
 'beer/style': 'Rauchbier',
 'review/palate': 3.5,
 'review/taste': 3.5,
 'beer/name': 'Rauch Ã\x9cr Bock',
 'review/timeUnix': 1297290394,
 'beer/ABV': 7.4,
 'beer/beerId': '58046',
 'beer/brewerId': '1075',
 'review/timeStruct': {'isdst': 0,
  'mday': 9,
  'hour': 22,
  'min': 26,
  'sec': 34,
  'mon': 2,
  'year': 2011,
  'yday': 40,
  'wday': 2},
 'review/overall': 3.5,
 'review/text': "Got this in a 22 ounce bomber. It poured a deep bright brown with a pretty small head. \t\tAromas of smoked kielba, and dark roasted malts. \t\tThis beer was OK but for a style that I like so much it did'nt really please as much as I thought it would. It had a nice amount of smokiness to it. Really obvious and pretty strong but not charred. But the oily consistency of the beer along with the huge saltiness made the beer less than I was hoping for. \t\tIt was still a beer that I'm glad I grabbed. They don't make many in the style anyways.",
 'user/profileName': 'cooch69',
 

In [3]:
review_length = [len(d['review/text']) for d in data if 'user/gender' in d]
def feature(datum):
    feat = [[1, d] for d in datum]
    return feat   
feat = feature(review_length)

In [4]:
gender = [d['user/gender'] for d in data if 'user/gender' in d]

In [5]:
def encoder(datum):
    encoded = []
    for i in range(len(datum)):
        if datum[i] == 'Female':
            encoded.append(1)
        else:
            encoded.append(0)
    return encoded

gender = encoder(gender)

In [6]:
#Logistic regression classifier
mod = linear_model.LogisticRegression(C=1.0)
mod.fit(feat, gender)

LogisticRegression()

In [7]:
train_predictions = mod.predict(feat)

In [8]:
train_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
def num_TP(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 0 and predict[i] == 0:
                count = count + 1
    return count
number_true_positive = num_TP(gender, train_predictions)
print('number of true positive:', number_true_positive)

number of true positive: 20095


In [10]:
def num_TN(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 1 and predict[i] == 1:
            count = count + 1
    return count
number_true_negative = num_TN(gender, train_predictions)
print('number of true negative:', number_true_negative)

number of true negative: 0


In [11]:
def num_FP(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 1 and predict[i] == 0:
            count = count + 1
    return count
number_false_positive = num_FP(gender, train_predictions)
print('number of false positive:', number_false_positive)

number of false positive: 308


In [12]:
def num_FN(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 0 and predict[i] == 1:
            count = count + 1
    return count
number_false_negative = num_FN(gender, train_predictions)
print('number of false negative:', number_false_negative)

number of false negative: 0


In [13]:
false_positive_rate = number_false_positive / (number_false_positive + number_true_negative)
print('false positive rate: ', false_positive_rate)

false positive rate:  1.0


In [14]:
false_negative_rate = number_false_negative / (number_false_negative + number_true_positive)
print('false negative rate: ', false_negative_rate)

false negative rate:  0.0


In [15]:
print('Balanced error rate: ', 0.5*(false_positive_rate + false_negative_rate))

Balanced error rate:  0.5


8. Retrain the regressor using the class weight=’balanced’ option, and report the same error metrics as above.

In [16]:
# Balanced model
mod = linear_model.LogisticRegression(C=1.0, class_weight='balanced')
mod.fit(feat, gender)
train_predictions = mod.predict(feat)

def num_TP(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 0 and predict[i] == 0:
                count = count + 1
    return count
number_true_positive = num_TP(gender, train_predictions)
print('number of true positive:', number_true_positive)

def num_TN(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 1 and predict[i] == 1:
            count = count + 1
    return count
number_true_negative = num_TN(gender, train_predictions)
print('number of true negative:', number_true_negative)

def num_FP(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 1 and predict[i] == 0:
            count = count + 1
    return count
number_false_positive = num_FP(gender, train_predictions)
print('number of false positive:', number_false_positive)

def num_FN(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 0 and predict[i] == 1:
            count = count + 1
    return count
number_false_negative = num_FN(gender, train_predictions)
print('number of false negative:', number_false_negative)

false_positive_rate = number_false_positive / (number_false_positive + number_true_negative)
print('false positive rate: ', false_positive_rate)

false_negative_rate = number_false_negative / (number_false_negative + number_true_positive)
print('false negative rate: ', false_negative_rate)

print('Balanced error rate: ', 0.5*(false_positive_rate + false_negative_rate))

number of true positive: 8423
number of true negative: 199
number of false positive: 109
number of false negative: 11672
false positive rate:  0.3538961038961039
false negative rate:  0.5808410052251803
Balanced error rate:  0.4673685545606421


9. Improve your predictor (i.e., reduce the balanced error rate) by incorporating additional features from the data (e.g. beer styles, ratings, features from text, etc.)

In [17]:
rate = [d['review/overall'] for d in data if 'user/gender' in d]
taste = [d['review/taste'] for d in data if 'user/gender' in d]

In [18]:
def feature(datum1, datum2, datum3):
    feat = [[1, d] for d in datum1]
    for i in range(len(datum2)):
        feat[i].append(datum2[i])
    for i in range(len(datum3)):
        feat[i].append(datum3[i])
    return feat
feat = feature(review_length, rate, taste)

In [19]:
# Balanced model
mod = linear_model.LogisticRegression(C=1.0, class_weight='balanced')
mod.fit(feat, gender)
train_predictions = mod.predict(feat)

def num_TP(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 0 and predict[i] == 0:
                count = count + 1
    return count
number_true_positive = num_TP(gender, train_predictions)
print('number of true positive:', number_true_positive)

def num_TN(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 1 and predict[i] == 1:
            count = count + 1
    return count
number_true_negative = num_TN(gender, train_predictions)
print('number of true negative:', number_true_negative)

def num_FP(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 1 and predict[i] == 0:
            count = count + 1
    return count
number_false_positive = num_FP(gender, train_predictions)
print('number of false positive:', number_false_positive)

def num_FN(actual, predict):
    count = 0
    for i in range(len(actual)):
        if actual[i] == 0 and predict[i] == 1:
            count = count + 1
    return count
number_false_negative = num_FN(gender, train_predictions)
print('number of false negative:', number_false_negative)

false_positive_rate = number_false_positive / (number_false_positive + number_true_negative)
print('false positive rate: ', false_positive_rate)

false_negative_rate = number_false_negative / (number_false_negative + number_true_positive)
print('false negative rate: ', false_negative_rate)

print('Balanced error rate: ', 0.5*(false_positive_rate + false_negative_rate))

number of true positive: 9808
number of true negative: 187
number of false positive: 121
number of false negative: 10287
false positive rate:  0.39285714285714285
false negative rate:  0.5119183876586215
Balanced error rate:  0.45238776525788216
